<h1> Implementing a CNN in a TensorFlow</h1>


In [ ]:
import warnings
warnings.filterwarnings('ignore')  

import numpy as np
import tensorflow as tf
from tensorflow.compat.v1.examples.tutorials.mnist import input_data
tf.logging.set_verbosity(tf.logging.ERROR)

import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow.compat.v1.examples'

In [ ]:
import tensorflow_datasets
(ds_train, ds_test), ds_info = tensorflow_datasets.load('mnist')


NameError: name 'tfds' is not defined

In [13]:
def initialize_weights(shape):
 return tf.Variable(tf.compat.v1.truncated_normal(shape, stddev=0.1))

def initialize_bias(shape):
 return tf.Variable(tf.compat.v1.constant(0.1, shape=shape))

def convolution(x, W):
 return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pooling(x):
 return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

X_ = tf.compat.v1.placeholder(tf.float32, [None, 784])
X = tf.reshape(X_, [-1, 28, 28, 1])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])


In [23]:
W1 = initialize_weights([5,5,1,32])
b1 = initialize_bias([32])
conv1 = tf.compat.v1.nn.relu(convolution(X, W1) + b1)
pool1 = max_pooling(conv1)

W2 = initialize_weights([5,5,32,64])
b2 = initialize_bias([64])
conv2 = tf.compat.v1.nn.relu(convolution(pool1, W2) + b2)
pool2 = max_pooling(conv2)

flattened = tf.reshape(pool2, [-1, 7*7*64])
W_fc = initialize_weights([7*7*64, 1024])
b_fc = initialize_bias([1024])
fc_output = tf.compat.v1.nn.relu(tf.compat.v1.matmul(flattened, W_fc) + b_fc)

W_out = initialize_weights([1024, 10])
b_out = initialize_bias([10])
YHat = tf.compat.v1.nn.softmax(tf.compat.v1.matmul(fc_output, W_out) + b_out)

In [24]:
cross_entropy = -tf.compat.v1.reduce_sum(y*tf.compat.v1.log(YHat))
optimizer = tf.compat.v1.train.AdamOptimizer(1e-4).minimize(cross_entropy)

predicted_digit = tf.argmax(YHat, 1)
actual_digit = tf.argmax(y, 1)

correct_pred = tf.equal(predicted_digit,actual_digit)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:


sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [35]:
for epoch in range(1000):
 #select some batch of data points according to the batch size (100)
 X_batch, y_batch = mnist.train.next_batch(batch_size=100)
 #train the network
 loss, acc, _ = sess.run([cross_entropy, accuracy, optimizer], feed_dict={X_: X_batch, y: y_batch})
 #print the loss on every 100th epoch
 if epoch%100 == 0:
  print('Epoch: {}, Loss:{} Accuracy: {}'.format(epoch,loss,acc))

AttributeError: 'dict' object has no attribute 'train'